# Grok-powered SBOM Analysis with AG2

This notebook demonstrates how to use AG2 (AutoGen) with Grok LLM to perform Software Bill of Materials (SBOM) analysis and security vulnerability scanning on Git repositories.


**Author:** Priyanshu Deshmukh

## Overview

The notebook showcases:

1. Setting up Grok LLM integration with AG2
2. Creating specialized security agents for vulnerability scanning
3. Using Trivy for automated SBOM analysis
4. Implementing function calling for repository scanning
5. Generating comprehensive security reports

## Requirements

```bash
pip install -q "ag2[openai,rag]"
```

## Ubuntu/Debian
   ```bash
   sudo apt-get install wget apt-transport-https gnupg lsb-release
   wget -qO - https://aquasecurity.github.io/trivy-repo/deb/public.key | sudo apt-key add -
   echo deb https://aquasecurity.github.io/trivy-repo/deb $(lsb_release -sc) main | sudo tee -a /etc/apt/sources.list.d/trivy.list
   sudo apt-get update
   sudo apt-get install trivy
   ```

In [ ]:
import os
import subprocess

from dotenv import load_dotenv

from autogen import AssistantAgent, ConversableAgent, LLMConfig

load_dotenv()

#### Configure Environment Variables
   ```
   XAI_API_KEY="your-grok-api-key"
   ```

In [ ]:
llm_config = LLMConfig(
    config_list=[
        {
            "model": "grok-4",
            "api_type": "openai",
            "base_url": "https://api.x.ai/v1",
            "api_key": os.getenv("XAI_API_KEY"),
        }
    ],
    temperature=0.0,
)

In [ ]:
with llm_config:
    appsec_agent = ConversableAgent(
        name="appsec_agent",
        system_message="You are a security expert who performs AppSec scans on git repositories.",
        human_input_mode="TERMINATE",
    )

    executer = AssistantAgent(
        name="executor_agent",
        human_input_mode="NEVER",
    )

## Function Definitions

### `scan_git_repo_vulnerabilities`

1. Performs automated vulnerability scanning on Git repositories using Trivy.

#### Parameters:
1. `git_repo_url` (str): The Git repository URL to scan  
2. `output_json_path` (Optional[str]): Path to save JSON output

#### Returns:
1. JSON string containing vulnerability scan results

In [ ]:
@appsec_agent.register_for_llm()
@executer.register_for_execution()
def scan_git_repo_vulnerabilities(
    git_repo_url: str, output_json_path: str | None = "../test/agentchat/contrib/graph_rag/scan.json"
) -> str:
    """
    Execute Trivy repo scan on a git repository and save the JSON output to a file.

    Args:
        git_repo_url (str): The git repository URL to scan
        output_json_path (Optional[str]): Path to save the JSON output. If None, does not save.

    Returns:
        str: The JSON output from Trivy scan as a string

    Raises:
        subprocess.CalledProcessError: If the Trivy command fails
        FileNotFoundError: If Trivy is not installed or not in PATH
    """
    try:
        # Execute the trivy command
        result = subprocess.run(
            ["trivy", "repo", "--format=json", git_repo_url], capture_output=True, text=True, check=True
        )
        json_output = result.stdout
        if output_json_path:
            with open(output_json_path, "w", encoding="utf-8") as f:
                f.write(json_output)
        return f"Scan result saved to {output_json_path} for : {git_repo_url} \n {json_output}"
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"
    except FileNotFoundError:
        return "Error: Trivy command not found. Please ensure Trivy is installed and in your PATH."

In [ ]:
appsec_agent.initiate_chat(
    message="Scan the following git repository for vulnerabilities and give me an analysis report: https://github.com/ine-labs/AWSGoat.git",
    max_turns=5,
    recipient=executer,
).process()